In [1]:
import math
from math import factorial
import sys

sys.path.append('../src/utils')  # Ensure this path points to the directory containing your utility modules

from po_fun import BasicUtils, StatisticalUtils,GenerationUtils,ConversionUtils
from po_accelerator import LogLikelihoodCache 


In [2]:
import math
import numpy as np

def mallows_local_factor(i, theta, y):

    if len(y) <= 1:
        return 1.0  # If there's only one item, the local factor is 1

    # Count how many items in `y` are positioned before `i` in `y` 
    # even though `i` has a smaller index in `y`.
    idx_i = y.index(i)
    d = 0
    for a in y:
        if a != i:
            if idx_i > y.index(a):
                d += 1

    # The denominator is sum_{k=0..(|y|-1)} e^{-theta * k}, for length(y) items
    n = len(y)
    denom = 0.0
    for k in range(n):
        denom += math.exp(-theta * k)

    numerator = math.exp(-theta * d)
    return numerator / denom


def is_total_order(adj_matrix: np.ndarray) -> bool:

    n = adj_matrix.shape[0]
    # Compute transitive closure
    closure = BasicUtils.transitive_closure(adj_matrix)

    # For every pair (i,j), i != j, check comparability
    for i in range(n):
        for j in range(n):
            if i != j:
                # Must have either i->j or j->i
                if not (closure[i, j] or closure[j, i]):
                    return False
    
    return True


def p_mallows_of_l_given_y(l, y, theta):
    """
    A basic function for p^{(M)}(l | y, theta) if we define it as a product of local factors
    or use the exponential form. We'll do local factor approach:
       p^{(M)}(l|y,theta) = ∏_{i=1..n} q^{(M)}(l_i | y_{i..n}, theta).
    We'll do a sequential interpretation: 
       we build l in order, at step i pick l[i], multiply local factor.
    """
    if len(l) != len(y):
        # if we interpret partial, or mismatch => return 0
        return 0.0

    prob_val = 1.0
    remain = list(y)  # items that haven't been "used" yet
    for item in l:
        if item not in remain:
            return 0.0
        # local factor
        gf = mallows_local_factor(item, theta,remain)
        prob_val *= gf
        # remove 'item'
        remain.remove(item)

    return prob_val


def f_mallows(y, h, theta, O_i_indice):
    """
    We pass 'O_i_indice' as the list of item labels corresponding
    to the rows/columns in 'h'. So h[i,:] corresponds to O_i_indice[i].
    """
    n = h.shape[0]
    # 1) If total => exactly 1 extension => p(l|y,theta)
    if is_total_order(h):
        # single extension
        l = GenerationUtils.topological_sort(h)
        # note that 'l' is in 0..n-1, but we interpret them as O_i_indice[l[i]]
        # let's convert that to actual labels
        real_l = [O_i_indice[x] for x in l]
        p_val = p_mallows_of_l_given_y(real_l, y, theta)
        return p_val

    # 2) If h is empty => sum=1
    if np.sum(h) == 0:
        return 1.0

    # 3) Sum over 'tops'
    tops = BasicUtils.find_tops(h)
    f_val = 0.0
    for t in tops:
        k_label = O_i_indice[t]    # the actual item label
        # local factor
        gk = mallows_local_factor(k_label, theta, y)

        # remove row t, col t
        h_sub = np.delete(np.delete(h, t, axis=0), t, axis=1)
        # remove item from O_i_indice
        O_i_sub = O_i_indice[:t] + O_i_indice[t+1:]

        # remove k_label from y
        y_sub = [itm for itm in y if itm != k_label]

        f_k = f_mallows(y_sub, h_sub, theta, O_i_sub)
        f_val += gk * f_k

    return f_val


def compute_mallows_likelihood(y, h, theta, O_i_indice):
    """
    p^{(M)}(y | h, theta) = f / count
    """
    f_val = f_mallows(y, h, theta, O_i_indice)
    tr_h = BasicUtils.transitive_reduction(h)
    c_val = BasicUtils.nle(tr_h)
    if c_val == 0:
        return 0.0
    return f_val / c_val

In [3]:
if __name__ == "__main__":
    # Let's define a partial order on 5 items labeled [10,20,30,40,50].
    # We'll store them in adjacency matrix h of size 5x5. row=col i => item i in O_i_indice
    # Suppose O_i_indice = [10,20,30,40,50].
    # We'll define edges: 10->20, 10->30, 30->40, 40->50 (so partial, but quite linear).
    # That adjacency means:
    # row=0 => item=10 has edges to col=1(item20), col=2(item30).
    # row=2 => item=30 has edge to col=3(item40).
    # row=3 => item=40 has edge to col=4(item50).

    O_i_indice = [10, 20, 30, 40, 50]  # item labels

    # adjacency for these 5 items
    h = np.array([
        [0,1,1,0,0],  # 10->20, 10->30
        [0,0,0,0,0],  # 20 no edges
        [0,0,0,1,0],  # 30->40
        [0,0,0,0,1],  # 40->50
        [0,0,0,0,0]   # 50 no edges
    ], dtype=int)

    # Observed ranking y
    y = [10,30,20,40,50]
    # Let's pick theta
    theta = 2.0

    # compute the Mallows likelihood
    likelihood = compute_mallows_likelihood(y, h, theta, O_i_indice)

    print("=== Test Sample ===")
    print(f"Item labels (O_i_indice) = {O_i_indice}")
    print("Adjacency matrix h:\n", h)
    print(f"Observed ranking y = {y}")
    print(f"theta = {theta}")
    print(f"Mallows partial-order likelihood = {likelihood}")

=== Test Sample ===
Item labels (O_i_indice) = [10, 20, 30, 40, 50]
Adjacency matrix h:
 [[0 1 1 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
Observed ranking y = [10, 30, 20, 40, 50]
theta = 2.0
Mallows partial-order likelihood = 0.18401390115966787
